# **Exporting distances of edges from given road segments**

In [1]:
MAX_DISTANCE = 150

In [2]:
import os, sys
import numpy as np
import json

SUMO_HOME = os.environ["SUMO_HOME"] #locating the simulator
sys.path.append(SUMO_HOME+"/tools")
import traci
import sumolib

In [1]:
MOST_PATH = "../build_inputs//MoSTScenario/scenario/most.sumocfg"
PARKING_AREA_DEFINITIONS = "../build_inputs/MoSTScenario/scenario/in/add/most.parking.add.xml"

In [4]:
import pandas as pd

parking_df = pd.read_xml(PARKING_AREA_DEFINITIONS)
parking_df = parking_df[parking_df["roadsideCapacity"]>0]

In [5]:
sumoBinary = "sumo"
sumoCmd = [sumoBinary, "-c", MOST_PATH,
           "--random", "--random-depart-offset", "900", "--verbose", "false",
           "--net-file", "../../MoSTScenario/scenario/in/most.net.xml"]
sumoCmd += ["--additional-files", "../../MoSTScenario/scenario/in/add/most.poly.xml,../../MoSTScenario/scenario/in/add/most.parking.allvisible.add.xml"]

traci.start(sumoCmd)

 Retrying in 1 seconds


sumo: /home/madbiker/anaconda3/envs/tf_env/lib/libtiff.so.5: no version information available (required by /lib/x86_64-linux-gnu/libFOX-1.6.so.0)


(20, 'SUMO 1.15.0')

In [6]:
parking_ids = parking_df["id"].values.tolist()
parking_lanes = parking_df["lane"].values.tolist()
parking_edges = []
for i in range(len(parking_lanes)):
    parking_edges.append(traci.lane.getEdgeID(parking_lanes[i]))

In [7]:
edges = traci.edge.getIDList()

distances = {}

for i in range(len(parking_ids)):
    for j in range(len(edges)):
        distance_ = traci.simulation.getDistanceRoad(parking_edges[i],
            traci.lane.getLength(parking_lanes[i])/2,
            edges[j],
            traci.lane.getLength(edges[j]+"_0")/2)
        if distance_ <= MAX_DISTANCE:
            if not(parking_ids[i] in distances):
                distances[parking_ids[i]] = []
            distances[parking_ids[i]].append(edges[j])

In [8]:
with open("../02_data/proximity.json", "w") as f:
    json.dump(distances, f)

In [9]:
traci.close()

Step #14400.00 (0ms ?*RT. ?UPS, TraCI: 172320ms, vehicles TOT 0 ACT 0 BUF 0)              


# **Utilizing the distances to create measurement files**

## Downscale measurement files

Resampling by 0.1 Hz

In [2]:
meas_root = "../dockeroutput/"

In [25]:
for day in range(60):
    meas_path = meas_root+"day_%d/"%day
    movements = pd.read_csv(meas_path+"_vehicle_positions.csv")
    movements = movements[movements["timestamp"]%10 == 0]
    movements.to_csv(meas_path+"ds_vehicle_positions.csv", index=False)

## Creating Measurement files

In [3]:
DAY = 0
MEASUREMENT_PATH = "../dockeroutput/day_%d/"%DAY

THREADS = 16

In [4]:
import os, sys
import numpy as np
import json
import pandas as pd

from multiprocessing.pool import Pool

In [5]:
parking_df = pd.read_xml(PARKING_AREA_DEFINITIONS).set_index("id")
parking_df = parking_df[parking_df["roadsideCapacity"]>0]

In [6]:
with open("../02_data/proximity.json", "r") as f:
    distances = json.load(f)
    
proximity = {}
for x in distances:
    for y in distances[x]:
        if not(y in proximity):
            proximity[y] = []
        proximity[y].append(x)

In [7]:
from tqdm.notebook import trange

def convert_measurement(day):
    '''supports multithreaded measurement conversion
        params:
            day: (int) measurement day
    '''
    
    meas_path = meas_root
    meas_path = meas_root+"day_%d/"%day
    
    #reading files:
    with open(meas_path+"edge_maps.json", "r") as f:
        maps = json.load(f)
        edge_to_idx = maps["edge_to_idx_map"]
        idx_to_edge = maps["idx_to_edge_map"]
    with open(meas_path+"vehicle_maps.json") as f:
        maps = json.load(f)
        idx_to_veh = maps["idx_to_vehicle_map"]
    movements = pd.read_csv(meas_path+"ds_vehicle_positions.csv")
    occupancies = pd.read_csv(meas_path+"occupancies.csv")

    #preprocessing occupancy data:
    percent_val = []
    for i,r in occupancies.iterrows():
        percent_val.append(r.counts / parking_df.loc[int(r.ids)]["roadsideCapacity"])
    occupancies["percentage"] = percent_val
    occupancies = occupancies.drop_duplicates()
    occupancies = occupancies.set_index(["ids", "timestamp"])
    
    #measurement:
    #ids = []
    #days = []
    #timestamps = []
    #p_lots = []
    #measured_occupancies = []
    max_debug = 0
    with open(meas_path+"vehicle_measurement.csv", "w") as f:
        f.write("veh_id,day,timestamp,parking_id,occupancy\n")
        for i in range(len(movements)):
            r = movements.iloc[i]
            if (r.timestamp%3600 == 0) and (r.timestamp//3600>max_debug):
                print(day,":\t",r.timestamp//3600)
                max_debug = r.timestamp//3600
            edge_id = idx_to_edge[str(r["position"])]
            if edge_id in proximity:
                for p_lot in proximity[edge_id]:
                    time = int((r.timestamp // 60)*60)
                    occupancy = occupancies.loc[int(p_lot), time]["percentage"]
                    #ids.append(idx_to_veh[str(r.ids)])
                    #days.append(day)
                    #timestamps.append(r.timestamp)
                    #p_lots.append(int(p_lot))
                    #measured_occupancies.append(occupancy)
                    f.write("%s,%d,%d,%d,%f\n"%(
                        idx_to_veh[str(r.ids)], day, r.timestamp, int(p_lot), occupancy
                    ))
    #del(movements)
    #results = pd.DataFrame([ids,days,timestamps,p_lots,measured_occupancies],
    #                      columns=["ids", "days", "timestamps", "p_lots", "occupancies"])
    #results.to_csv(meas_path+"vehicle_measurement.csv", index=False)

In [8]:
days = [i for i in range(60)]

pool = Pool(processes=THREADS)
pool.map(convert_measurement, days)

1 :	 36 :	 
6
2 :	 6
14 :	 6
15 :	 6
11 :	 6
8 :	 6
13 :	 6
12 :	 6
4 :	 6
10 :	 6
5 :	 6
9 :	 6
6:	  6
7 :	 6
0 :	 6
3 :	 7
14 :	 7
2 :	 7
1 :	 7
15 :	 7
11 :	 7
13 :	 7
8 :	 7
12 :	 7
5 :	 7
4 :	 7
9 :	 7
6:	  7
7 :	 7
10 :	 7
0 :	 7
14 :	 8
3 :	 8
2 :	 8
1 :	 8
11 :	 8
15 :	 8
13 :	 8
8 :	 8
4 :	 8
5 :	 8
12 :	 8
7 :	 8
9 :	 8
6 :	 8
10 :	 8
0 :	 8
14 :	 9
3 :	 9
2 :	 9
11 :	 9
1 :	 9
15 :	 9
13 :	 9
8 :	 9
4 :	 9
5 :	 9
12 :	 9
10 :	 9
7 :	 9
6 :	 9
9 :	 9
0 :	 9
3 :	 10
2 :	 10
1 :	 10
11 :	 10
14 :	 10
15 :	 10
4 :	 10
8 :	 10
10 :	 10
0 :	 10
13 :	 10
5:	  10
12 :	 10
7 :	 10
6 :	 10
9 :	 10
3 :	 11
2 :	 11
1 :	 11
11 :	 11
4 :	 11
14 :	 11
10 :	 11
8 :	 11
15 :	 11
0 :	 11
13 :	 11
5 :	 11
12 :	 11
7 :	 11
6 :	 11
9 :	 11
3 :	 12
1:	  12
2 :	 12
11 :	 12
10 :	 12
8 :	 12
0 :	 12
4 :	 12
14 :	 12
15:	  12
5 :	 12
7 :	 12
13 :	 12
12 :	 12
6 :	 12
9 :	 12
3 :	 13
1 :	 13
2 :	 13
11 :	 13
8 :	 13
10 :	 13
4 :	 13
0 :	 13
15 :	 13
14 :	 13
5 :	 13
7 :	 13
13 :	 13
12 :	 13
6 :	 13


[None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None]